In [26]:
import matplotlib.pylab as plt

import numpy as np

In [4]:
EPSILON = 1e-8

In [5]:
def plane_intersection(ray, plane):
    ''' Find the intersection point between a ray and a plane
            ray: tuple (A, u) starting point and direction
            plane: tuple (P, n, ux) origin, normal direction and in-plane x-axis
            
        Returns:
            coordinates of the intersection point in the 3D ref. frame
            coordinates of the intersection point in the plane ref. frame (u, v)
    '''
    A, u = ray
    P, n, plane_x = plane
    
    A, P = np.asarray(A), np.asarray(P)
    u, n, plane_x = np.asarray(u), np.asarray(n), np.asarray(plane_x)
    
    u_dot_n = np.inner(u, n)
    AP_dot_n = np.inner(P - A, n)
    
    if u_dot_n >= -EPSILON and AP_dot_n >= -EPSILON:    
        # print('do not colide')
        return None, None
    else:
        t = AP_dot_n / u_dot_n  # time of collision
        B = A + u*t   # collision point
        
        plane_y = np.cross(n, plane_x)
        proj_x = np.inner(plane_x, B-P)
        proj_y = np.inner(plane_y, B-P)
        return B, (proj_x, proj_y)

# test
A = (1.2, 0, 0)
u = (-1, 0, 0)
ray = (A, u)

P = (0, 0, 0)
n = (1, 1, 4)
plane_x = (-1, 1, 0)
plane = (P, n, plane_x)

assert np.allclose(plane_intersection(ray, plane)[0], [0., 0., 0.])


In [106]:
N = 1000
A = np.random.rand(N, 3)
u = .1 * np.random.rand(N, 3)
u[:, 0] += -.08
A[:, 0] += 3

plane_center = np.array((0, 0, 0))
plane_normal = np.array((1, 0, 0)) 

AP_dot_n = np.inner((A - plane_center), plane_normal)

u_dot_n = np.inner(u, plane_normal)

t = - AP_dot_n / u_dot_n
B = A + u*t[:, np.newaxis]

In [107]:
B[t<0] = np.NaN

# Go

In [343]:
def norm(array_of_vector):
    ''' return the array of normed vector (vector along the last dim)
        i.e. a (nbre_vector, n_dim) array'''
    array_of_vector = np.asarray(array_of_vector)
    norm = np.linalg.norm(array_of_vector, axis=-1, keepdims=True)
    return  np.divide(array_of_vector, norm, where=norm>0)

# test
assert np.allclose(norm(np.array((2, 0, 0))),
                   np.array((1, 0, 0)))
assert np.allclose(norm(np.array([(0, 2, 0), (0, -3, 0), (0, 0, 0), (-1, 0, 0)])),
                   np.array([(0, 1, 0), (0, -1, 0), (0, 0, 0), (-1, 0, 0)]))

In [350]:
# inputs
N = 5
A = np.random.rand(N, 3)
u_incident = .1 * np.random.rand(N, 3)
u_incident[:, 0] += -.08
A[:, 0] += 3

# Define detector
deuxtheta = np.pi/4
rayon_gonio = 420 # mm
detector_position  = np.array((-rayon_gonio*np.cos(deuxtheta),
                              0,
                              +rayon_gonio*np.sin(deuxtheta)))
detector_normal = norm(np.array((+np.cos(deuxtheta),
                           0,
                           -np.sin(deuxtheta))))
detector_vertical = norm(np.array((0, 1, 0)))

# Define sample
sample_position  = norm((0, 0, 0))
sample_normal = norm((+np.cos(deuxtheta/2), 0, -np.sin(deuxtheta/2)))


In [347]:
# Intersection Incident beams with sample plane:
AP_dot_n = np.inner((A - sample_position), sample_normal)

u_dot_n = np.inner(u_incident, sample_normal)

t = - AP_dot_n / u_dot_n
B = A + u_incident*t[:, np.newaxis]

In [351]:
v_plane = np.cross(detector_normal, u_incident)
u_plane = np.cross(v_plane , detector_normal)

beta = np.arccos( np.dot(u_incident, detector_normal) )

In [352]:
beta

array([1.6303903 , 1.63847434, 1.59954499, 1.63768754, 1.59713146])

$$rho(\Gamma) = \frac{tan_{\alpha} \left(tan_{\alpha} \sin{\left(\beta \right)} \cos{\left(\Gamma \right)} - \sqrt{\sin^{2}{\left(\Gamma \right)} + \cos^{2}{\left(\Gamma \right)} \cos^{2}{\left(\beta \right)}}\right)}{- tan_{\alpha}^{2} \sin^{2}{\left(\beta \right)} \cos^{2}{\left(\Gamma \right)} + \sin^{2}{\left(\Gamma \right)} + \cos^{2}{\left(\Gamma \right)} \cos^{2}{\left(\beta \right)}}$$
